# dfply 기초

## 실습 데이터셋 준비

In [ ]:
# 관련 라이브러리를 호출합니다.
import os
from dfply import *

In [ ]:
# 현재 작업경로를 확인합니다.
os.getcwd()

In [ ]:
# 작업경로를 data 폴더로 변경합니다.
os.chdir('../data')

In [ ]:
# 현재 경로에 포함된 폴더명과 파일명을 출력합니다.
os.listdir()

## 컬럼 선택 및 제거

In [ ]:
# xlsx 파일을 읽고 데이터프레임을 생성합니다.
kbo = pd.read_excel('KBO_Hitters_2020.xlsx')

In [ ]:
# 데이터프레임에서 필요한 컬럼명을 지정하여 해당 열을 선택합니다.
kbo >> select(X['팀명'], X['선수명'], X['경기'])

In [ ]:
# 컬럼명 대신 정수 인덱스를 나열해도 됩니다.
kbo >> select(1, 0, 2)

In [ ]:
# 컬럼명 앞에 ~ 기호를 추가하면 삭제합니다.
kbo >> select(~X['타율'], ~X['OPS'])

In [ ]:
# 데이터프레임에서 불필요한 열을 제거합니다.
kbo >>= drop(X['타율'], X['OPS'])

## 조건에 맞는 행 걸러내기

In [ ]:
# kbo에서 홈런이 30 이상인 행만 선택합니다.
kbo >> mask(X['홈런'] >= 30)

In [ ]:
# kbo에서 안타가 150 이상인 행만 선택합니다.
kbo >> mask(X['안타'] >= 150)

In [ ]:
# kbo에서 타점이 100 이상인 행만 선택합니다.
kbo >> mask(X['타점'] >= 100)

In [ ]:
# 세 가지 조건을 모두 만족하는 행을 선택합니다.
kbo >> mask(X['홈런'] >= 30, X['안타'] >= 150, X['타점'] >= 100)

In [ ]:
# pandas 방식으로 세 가지 조건을 만족하는 행을 선택합니다.
kbo[(kbo['홈런'] >= 30) & (kbo['안타'] >= 150) & (kbo['타점'] >= 100)]

## 집계함수로 데이터 요약

In [ ]:
# 팀별 안타 합계 및 타수 합계 컬럼을 생성하여 데이터프레임을 반환합니다.
teamStat = kbo \
>> group_by(X['팀명']) \
>> summarize(팀타수 = X['타수'].sum(), 
             팀안타 = X['안타'].sum())

In [ ]:
# teamStat의 정보를 확인합니다.
teamStat.info()

In [ ]:
# teamStat을 출력합니다.
teamStat

## 파생변수 생성

In [ ]:
# teamStat에서 팀타율 컬럼을 생성합니다.
teamStat >>= mutate(팀타율 = (X['팀안타'] / X['팀타수']).round(3))

In [ ]:
# teamStat을 출력합니다.
teamStat

## 데이터 정렬

In [ ]:
# teamStat에서 팀타율 기준으로 내림차순 정렬합니다.
teamStat >>= ungroup() >> arrange(X['팀타율'], ascending = False)

In [ ]:
# teamStat을 출력합니다.
teamStat

## [참고] 그룹 & 집계 & 파생변수 & 정렬을 한 번에 실행

In [ ]:
# kbo에서 팀별 팀안타, 팀타수 컬럼을 생성하고 팀타율을 계산하여 반환합니다.
kbo \
>> group_by(X['팀명']) \
>> summarize(팀타수 = X['타수'].sum(), 팀안타 = X['안타'].sum()) \
>> mutate(팀타율 = (X['팀안타'] / X['팀타수']).round(3)) \
>> ungroup() \
>> arrange(X['팀타율'], ascending = False)

# 기술통계 분석

## 실습 데이터셋 준비

In [ ]:
# 관련 라이브러리를 호출합니다.
import numpy as np
import pandas as pd
import chardet

In [ ]:
# 현재 작업경로에 포함된 폴더명과 파일명을 출력합니다.
os.listdir()

In [ ]:
# 실습할 csv 파일명을 fileName에 할당합니다.
fileName = 'APT_List_Seoul_2020.csv'

In [ ]:
# csv 파일을 'bytes'로 읽습니다.
raw = open(fileName, 'rb').read()

In [ ]:
# 텍스트 파일의 문자 인코딩 방식을 확인합니다.
chardet.detect(raw[:100])

In [ ]:
# 아파트 csv 파일을 읽고 데이터프레임을 생성합니다.
apt = pd.read_csv(fileName)

In [ ]:
# apt의 정보를 확인합니다.
apt.info()

In [ ]:
# 거래일 컬럼의 자료형을 'datetime64'로 변환합니다.
apt['거래일'] = pd.to_datetime(apt['거래일'])

In [ ]:
# apt의 컬럼별 자료형을 출력합니다.
apt.dtypes

## 대푯값 : 평균, 절사평균

In [ ]:
# 거래금액 컬럼의 평균을 반환합니다.
apt['거래금액'].mean()

In [ ]:
# scipy.stats 모듈을 호출합니다.
from scipy import stats

In [ ]:
# 양 극단에서 10%씩 제외한 평균을 계산합니다.
stats.trim_mean(apt['거래금액'], 0.1)

## 대푯값 : 중위수

In [ ]:
# 거래금액 컬럼의 중위수를 반환합니다.
apt['거래금액'].median()

In [ ]:
# [주의] 양 극단의 50%씩 절사한 평균을 계산하면 원소 개수에 따라 중위수 또는 결측값을 반환합니다.
stats.trim_mean(apt['거래금액'], 0.5)

In [ ]:
# 원소 개수가 홀수이면 결측값 대신 중위수를 반환합니다.
stats.trim_mean(range(11), 0.5)

## 대푯값 : 최빈값

In [ ]:
# 시군구 컬럼의 최빈값을 반환합니다.
apt['시군구'].mode()

In [ ]:
# 라이브러리를 호출합니다.
from dfply import *

In [ ]:
# 집계함수를 이용하여 시군구별 거래건수를 확인합니다.
apt \
>> group_by(X['시군구']) \
>> summarize(거래건수 = X['시군구'].count()) \
>> ungroup() \
>> arrange(X['거래건수'], ascending = False)

## 분포 : 범위

In [ ]:
# 거래금액 컬럼의 최소값을 반환합니다.
apt['거래금액'].min()

In [ ]:
# 거래금액 컬럼의 최대값을 반환합니다.
apt['거래금액'].max()

In [ ]:
# 거래금액 컬럼의 범위를 반환합니다.
apt['거래금액'].max() - apt['거래금액'].min()

## 분포 : 분위수

In [ ]:
# 거래금액 컬럼의 10 백분위수를 반환합니다.
apt['거래금액'].quantile(0.10)

In [ ]:
# 거래금액 컬럼의 90 백분위수를 반환합니다.
apt['거래금액'].quantile(0.90)

In [ ]:
# 거래금액 컬럼의 10, 90 백분위수를 함께 반환합니다.
apt['거래금액'].quantile([0.10, 0.90])

In [ ]:
# 거래금액 컬럼의 십분위수를 반환합니다.
apt['거래금액'].quantile(np.arange(0.0, 1.1, 0.1))

## 분포 : 사분위수와 사분범위

In [ ]:
# 거래금액 컬럼의 1 사분위수를 반환합니다.
apt['거래금액'].quantile(0.25)

In [ ]:
# 거래금액 컬럼의 3 사분위수를 반환합니다.
apt['거래금액'].quantile(0.75)

In [ ]:
# 거래금액 컬럼의 최소값, 최대값 및 사분위수를 함께 반환합니다.
apt['거래금액'].quantile(np.arange(0.0, 1.1, 0.25))

In [ ]:
# 거래금액 컬럼의 사분범위를 계산합니다.
apt['거래금액'].quantile(0.75) - apt['거래금액'].quantile(0.25)

## 분포 : 분산

In [ ]:
# 거래금액 컬럼의 분산을 반환합니다.
apt['거래금액'].var()

## 분포 : 표준편차

In [ ]:
# 거래금액 컬럼의 표준편차를 반환합니다.
apt['거래금액'].std()

## 분포 : 중위수절대편차

In [ ]:
# 거래금액 컬럼의 중위수절대편차를 반환합니다. (값이 조금 다릅니다!)
apt['거래금액'].mad()

In [ ]:
# 로버스트한 통계량을 반환하는 모듈을 호출합니다.
from statsmodels import robust

In [ ]:
# 거래금액 컬럼의 중위수절대편차를 반환합니다.
robust.mad(apt['거래금액'])

## 여러 컬럼의 기술통계량 생성

In [ ]:
# apt의 숫자 컬럼에 대한 평균을 반환합니다.
apt.apply('mean', numeric_only = True, axis = 0)

In [ ]:
# apt의 숫자 컬럼에 대한 분산을 반환합니다.
apt.apply('var', numeric_only = True, axis = 0)

In [ ]:
# apt의 숫자 컬럼에 대한 표준편차를 반환합니다.
apt.apply('std', numeric_only = True, axis = 0)

In [ ]:
# apt의 문자열 컬럼에 대한 최빈값을 반환합니다.
apt.apply('mode', axis = 0)

In [ ]:
# apt의 모든 숫자 컬럼에 대한 기술통계량을 반환합니다.
apt.describe()

In [ ]:
# apt의 모든 문자열 컬럼에 대한 기술통계량을 반환합니다.
apt.describe(include = 'object')

## 상관관계 : 공분산

In [ ]:
# 전용면적과 거래금액 컬럼 간 공분산을 반환합니다.
apt['전용면적'].cov(apt['거래금액'])

In [ ]:
# apt의 모든 숫자 컬럼 간 공분산 행렬을 반환합니다.
apt.cov()

## 상관관계 : 상관계수

In [ ]:
# 전용면적과 거래금액 컬럼 간 상관계수를 반환합니다.
apt['전용면적'].corr(apt['거래금액'])

In [ ]:
# apt의 모든 숫자 컬럼 간 상관계수 행렬을 반환합니다.
apt.corr()

# 데이터 시각화

## 관련 라이브러리 호출 및 기본 설정

In [ ]:
# 관련 라이브러리를 호출합니다.
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [ ]:
# 마이너스 기호가 제대로 출력되도록 설정합니다.
mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
# 그래프의 크기와 해상도를 설정합니다.
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

In [ ]:
# 경고문구가 출력되지 않도록 설정합니다.
import warnings
warnings.filterwarnings('ignore')

## 한글폰트 설정

In [ ]:
# 한글폰트가 설치된 경로를 찾습니다.
fontList = fm.findSystemFonts(fontext = 'ttf')

In [ ]:
# 한글폰트명으로 검색합니다.
[font for font in fontList if 'Nanum' in font]

In [ ]:
# 한글폰트를 설정합니다.
fontPath = '/Library/Fonts/NanumGothic.ttf'
fontProp = fm.FontProperties(fname = fontPath)

In [ ]:
# [참고] 그래프에서 한글이 네모로 출력되는 에러를 해결하는 방법을 소개합니다.
mpl.get_cachedir()

In [ ]:
# 위 코드를 실행해서 출력되는 폴더로 이동하여 tex.cache 폴더 및 fontList.json 파일을 삭제합니다.
# 주피터 노트북을 재실행하면 한글폰트가 제대로 설정됩니다.

## 색상 팔레트 설정

In [ ]:
# pyplot 모듈에서 제공되는 컬러맵 목록을 출력합니다.
dir(plt.cm)

In [ ]:
# 관심 있는 팔레트의 색상을 출력합니다.
sns.color_palette('CMRmap')

In [ ]:
# 출력할 색상의 개수를 설정합니다.
sns.color_palette('pastel', 10)

In [ ]:
# 관심 있는 컬러맵을 출력합니다.
sns.color_palette('Spectral', 10)

In [ ]:
# 관심 있는 컬러맵을 출력합니다.
sns.color_palette('Set2', 10)

## Seaborn 테마 설정

In [ ]:
# 그래프에 적용될 다양한 옵션을 설정합니다.
sns.set_theme(context = 'notebook', 
              style = 'white', 
              palette = 'Set2', 
              font = fontProp.get_name(),
              font_scale = 1.2)

## 도수분포표

In [ ]:
# 계급을 설정하기 위해 데이터의 최소값과 최대값을 미리 확인합니다.
apt['거래금액'].describe()

In [ ]:
# 도수분포표의 계급을 설정합니다.
bins = np.arange(0, 81, 5)

In [ ]:
# 계급을 출력합니다. (이상, 미만)
bins

In [ ]:
# 계급별 빈도수를 반환합니다.
freq, edge = np.histogram(apt['거래금액'], bins)

In [ ]:
# 상대도수를 계산합니다.
prop = (freq / len(apt['거래금액'])).round(4)

In [ ]:
# 도수분포표를 데이터프레임으로 생성하여 출력합니다.
pd.DataFrame([bins, freq, prop], index = ['계급', '도수', '상대도수']).T

## 히스토그램 그리기

In [ ]:
# 히스토그램은 일변량 연속형 데이터의 분포(도수분포표)를 시각화한 것입니다.
sns.histplot(data = apt, 
             x = '거래금액', 
             bins = bins,
             color = '0.5',
             edgecolor = 'black',)
plt.title('거래금액 히스토그램')
plt.show()

In [ ]:
# 시군구별 평균 거래금액을 계산하여 데이터프레임으로 생성합니다.
siggMean = apt \
>> group_by(X['시군구']) \
>> summarize(평균금액 = X['거래금액'].mean()) \
>> mutate(평균금액 = X['평균금액'].round(2)) \
>> ungroup() \
>> arrange(X['평균금액'], ascending = False)

In [ ]:
# 시군구별 평균 거래금액 데이터프레임의 인덱스를 초기화합니다.
siggMean = siggMean.reset_index(drop = True)

In [ ]:
# 시군구별 평균 거래금액을 처음 10행만 출력합니다.
siggMean.head(10)

In [ ]:
# 강남구, 서초구, 용산구, 송파구 등 4개 시군구만 선택하여 top4를 생성합니다.
top4 = apt >> mask(X['시군구'].str.contains('강남|서초|용산|송파'))

In [ ]:
# 상위 4개 자치구 데이터프레임의 정보를 확인합니다.
top4.info()

In [ ]:
# 특정 컬럼의 값에 따라 히스토그램을 여러 개 겹쳐서 그릴 수 있습니다.
sns.histplot(data = top4, 
             x = '거래금액', 
             hue = '시군구', 
             bins = 100,
             edgecolor = 'black')
plt.title('상위 4개 시군구별 거래금액 히스토그램')
plt.show()

In [ ]:
# 특정 컬럼의 값에 따라 히스토그램을 나누어서 그릴 수 있습니다.
sns.displot(data = top4, 
            x = '거래금액', 
            col = '시군구', 
            binwidth = 1,
            color = 'blue', 
            height = 3)
plt.show()

## 상자수염그림 그리기

In [ ]:
# 상자수염그림은 연속형 데이터의 사분위수와 이상치를 시각화한 것입니다.
sns.boxplot(y = '거래금액', 
            data = apt, 
            color = 'gold')
sns.despine(bottom = True)
plt.title('거래금액 상자수염그림')
plt.show()

In [ ]:
# 집단별 상자수염그림을 그리면 여러 집단의 분포를 한 눈에 비교할 수 있습니다.
sns.boxplot(x = '시군구', 
            y = '거래금액', 
            data = apt, 
            fliersize = 4)
plt.title('시군구별 거래금액 상자수염그림')
plt.xticks(rotation = 90)
plt.show()

## 일변량 막대그래프 그리기

In [ ]:
# 일변량 막대그래프는 명목형 데이터의 빈도수를 막대로 시각화한 것입니다.
sns.countplot(x = '시군구', 
              data = apt, 
              hue = '금액구분')
plt.title('시군구별 거래건수 막대그래프')
plt.xticks(rotation = 90)
plt.show()

## 이변량 막대그래프 그리기

In [ ]:
# 이변량 막대그래프는 명목형 데이터에 따라 연속형 데이터의 크기를 표현합니다.
sns.barplot(x = '시군구', 
            y = '평균금액', 
            data = siggMean)
plt.title('시군구별 평균 거래금액 막대그래프')
plt.xticks(rotation = 90)
plt.show()

In [ ]:
# 막대그래프 위에 숫자를 텍스트로 추가합니다.
sns.barplot(x = '시군구', y = '평균금액', data = siggMean)

for index, row in siggMean.iterrows():
    plt.text(x = index, y = row['평균금액'] + 0.1, s = row['평균금액'], 
             fontsize = 10, ha = 'center', va = 'bottom', c = 'black')

plt.title('시군구별 평균 거래금액 막대그래프')
plt.xticks(rotation = 90)
plt.ylim(0, 20)
plt.show()

## 선그래프 그리기

In [ ]:
# apt에 거래월 컬럼을 정수형 시리즈로 생성합니다.
apt['거래월'] = apt['거래일'].dt.month

In [ ]:
# 월별 거래금액의 평균을 선그래프로 그립니다.
sns.lineplot(x = '거래월', y = '거래금액', data = apt, ci = 95)
plt.title('월별 평균 거래금액 선그래프')
plt.show()

In [ ]:
# 선그래프에 점을 추가한 그래프를 그립니다.
sns.pointplot(x = '거래월', 
              y = '거래금액', 
              data = apt, 
              hue = '금액구분',
              palette = 'Spectral')
plt.title('금액구분에 따른 월별 평균 거래금액 선그래프')
plt.show()

In [ ]:
# 특정 컬럼의 값에 따라 선그래프를 겹쳐서 그릴 수 있습니다.
top4['거래월'] = top4['거래일'].dt.month
sns.lineplot(x = '거래월', 
             y = '거래금액', 
             data = top4, 
             hue = '시군구', 
             palette = 'Spectral', 
             size = '시군구')
plt.title('상위 4개 시군구의 월별 평균 거래금액 선그래프')
plt.show()

## 산점도 그리기

In [ ]:
# 산점도는 이변량 연속형 데이터 간 상관관계를 시각화한 것입니다.
sns.scatterplot(x = '전용면적', 
                y = '거래금액', 
                data = apt,
                hue = '금액구분',
                edgecolor = 'black', 
                linewidth = 0.2)
plt.title('전용면적과 거래금액의 관계')
plt.show()

In [ ]:
# 특정 컬럼의 값에 따라 산점도를 겹쳐서 그릴 수 있습니다.
sns.scatterplot(x = '전용면적', 
                y = '거래금액', 
                data = top4, 
                hue = '시군구', 
                palette = 'deep',
                edgecolor = 'white')
plt.title('상위 4개 시군구별 전용면적과 거래금액의 관계')
plt.show()

## 산점도 행렬 그리기

In [ ]:
# 산점도 행렬을 그릴 일부 숫자 컬럼을 선택합니다. (5~6개가 적당합니다.)
top4_1 = top4 >> select(['시군구', '거래금액', '전용면적', '층', '세대수', '용적률'])

In [ ]:
# 선택한 숫자 컬럼으로 산점도 행렬을 그립니다.
sns.pairplot(data = top4_1, hue = '시군구', palette = 'deep')
plt.show()

---